# Analysez vos données textuelles

In [1]:
!pip install pandas


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd

## Prétraitez les données textuelles

### Récupérer et explorer le corpus de texte

Un peu de vocabulaire :
- corpus : ensemble de documents regroupés dans un optique ou une thématique précise
- document : texte appartenant à un corpus, mais indépendant des autres textes. Peut-être consituté de plusieurs phrases ou plusieurs paragraphes
- token : mot ou élément de ponctuation
- vocabulaire : ensemble des tokens DISTINCTS présent dans le corpus

In [5]:
db = load_all_sentences();
print('chargement de {} vers dans la db'.format(len(db.keys())))

NameError: name 'load_all_sentences' is not defined

In [6]:
from collections import defaultdict
base_artistes = defaultdict(set)
for k,v in db.iteritems():
    base_artistes[v['artistes']].add(k)
artistes = { k:v for k,v in artistes.iteritems() if len(v) > 200 }
print('{} artistes'.format(len(artistes)))

NameError: name 'db' is not defined

In [7]:
import nltk
test = "Bonjour, je suis un texte d'exemple pour le cours d'Openclassrooms. Soyez attentifs à ce cours !"

nltk.word_tokenize(test)

ModuleNotFoundError: No module named 'nltk'

In [8]:
tokenizer = nltk.RegexpTokenizer(r'\w+')
tokenizer.tokenize("Bonjour, je suis un texte d'exemple pour le cours d'Openclassrooms. Soyez attentifs à ce cours !")

NameError: name 'nltk' is not defined

In [9]:
tokenizer = nltk.RegexpTokenizer(r'\w+')

def freq_stats_corpora():
    corpora = defaultdict(list)

    # Création d'un corpus de tokens par artiste
    for artiste,sentence_id in artistes.iteritems():
        for sentence_id in sentence_id:
            corpora[artiste] += tokenizer.tokenize(
                                    db[sentence_id]['text'].decode('utf-8').lower()
                                )

    stats, freq = dict(), dict()

    for k, v in corpora.iteritems():
        freq[k] = fq = nltk.FreqDist(v)
        stats[k] = {'total': len(v)} 
        
    return (freq, stats, corpora)

# Récupération des comptages
freq, stats, corpora = freq_stats_corpora()
df = pd.DataFrame.from_dict(stats, orient='index')

# Affichage des fréquences
df.sort(columns='total', ascending=False)
df.plot(kind='bar', color="#f56900", title='Top 50 Rappeurs par nombre de mots')

NameError: name 'nltk' is not defined

In [10]:
def freq_stats_corpora():
    corpora = defaultdict(list)
    for artiste,sentence_ids in artistes.iteritems():
        for sentence_id in sentence_ids:
            corpora[artiste] += tokenizer.tokenize(
                                    db[sentence_id]['text'].decode('utf-8').lower()
                                )
        
    stats, freq = dict(), dict()

    for k, v in corpora.iteritems():
        freq[k] = fq = nltk.FreqDist(v)
        stats[k] = {'total': len(v), 'unique': len(fq.keys())}

    return (freq, stats, corpora)

## TP

### Exploratory Data Analysis

Quelle est la forme du Dataframe ? 

Y a t-il des valeurs manquantes ou des valeurs dupliquées ? 

Quelles sont les colonnes qui vont nous intéresser ? 

Y a-t-il des données aberrantes ou des incohérences majeures dans les données ? 

Y a t-il des tweets anormalement longs / courts ? Peut-on les considérer comme des outliers ? 

Quel est le ratio tweet qui parlent de “catastrophes” / tweet normaux ?

En regardant quelques tweets au hasard, peut-on deviner facilement la “target” ? 

Peut-on déjà détecter des “patterns” ou des mots clés dans les tweets?

A votre avis quel serait l’accuracy score qu’un humain pourrait obtenir s’il prédisait  les données “à la main” ?

In [30]:
df = pd.read_csv('./data/TP_analyse_donnees_textuelles/train.csv')
df.sample(10)

,id,keyword,location,text,target
2493,3582,desolate,Macclesfield,@binellithresa TY for the follow Go To http://...,1
2835,4079,displaced,"Oakland, CA",Historic flooding across Asia leaves hundreds ...,1
7397,10584,windstorm,NaN,Texas Seeks Comment on Rules for Changes to Wi...,0
3340,4784,evacuated,NaN,#WorldNews Fallen powerlines on G:link tram: U...,1
1182,1702,bridge%20collapse,UK,Australia's Ashes disaster - how the collapse ...,1
3004,4317,dust%20storm,D(M)V,@RetiredFilth people in sydney woke up to the ...,1
6728,9642,thunderstorm,South Carolina,Strong Thunderstorm 4 Miles East of Pickens Mo...,1
2315,3324,demolished,NaN,Lol meerkat is fucked. They will get demolishe...,0
45,65,ablaze,NaN,I gained 3 followers in the last week. You? Kn...,0
1394,2012,casualties,NaN,Another movie theater attack..close to home th...,1


In [28]:
df['target'].value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [29]:
df['location'].value_counts()

location
USA                   104
New York               71
United States          50
London                 45
Canada                 29
                     ... 
Silesia, Poland         1
Hickville, USA          1
New York NYC            1
Valle Del Sol           1
todaysbigstock.com      1
Name: count, Length: 3341, dtype: int64

In [16]:
print(f'La forme du dataframe est {df.shape}')

La forme du dataframe est (7613, 5)


In [15]:
df.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [19]:
pd.unique(df)

/tmp/ipykernel_32752/656283662.py:1: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  pd.unique(df)


array(['id', 'keyword', 'location', 'text', 'target', None], dtype=object)

In [22]:
df.duplicated().sum()


np.int64(0)

In [21]:
df[df.duplicated()]

,id,keyword,location,text,target


Pas de valeurs dupliquées

Les colonnes qui vont nous intéresser sont keyword, location, text et target

Y a-t-il des données aberrantes ou des incohérences majeures dans les données ?  Aucune idée

Y a t-il des tweets anormalement longs / courts ? Peut-on les considérer comme des outliers ? 


## Transformez les données textuelles

## Détectez automatiquement les sentiments des commentaires clients